# HW_9

- ✅ Дизайн базы. Рекомендуется, например, такой: в базе данных три таблицы, где (1) таблица с вопросами анкеты; (2) таблица с полученными данными; (3) таблица с информацией об информанте (имя/возраст/город проживания/гендер/родной язык и т.п.); id таблиц (2) и (3) совпадают
- ✅ Таблицы и их связи описаны через классы sqlalchemy
- ✅ Данные сохраняются в базу и это делается через описанные классы
- ✅ Добавить на страницу статистики приложения из дз8 
 - 1) какую-нибудь статистику по метаданным (например, сколько человек прошло опрос, какой максимальный, минимальный и средний возраст информантов)
 - 2) какую-нибудь статистику по вопросам анкеты (например, медиана рейтинга каждого стимульного предложения).
- ⬜️Выложить приложение на pythonanywhere. Соответствующая ссылка должна быть в файле readme репозитория


In [92]:
from flask_sqlalchemy import SQLAlchemy
from flask import Flask, render_template, request
import sqlite3

import collections
import numpy as np
import matplotlib.pyplot as plt

### 1. Создаю базу

Дизайн базы: 
- (1) таблица с вопросами анкеты
- (2) таблица с полученными данными
- (3) таблица с информацией об информанте (имя/возраст/город проживания/гендер/родной язык и т.п.)
- id таблиц (2) и (3) совпадают

In [93]:
conn = sqlite3.connect('mansi_language.db')
cur = conn.cursor()

cur.execute("""
CREATE TABLE IF NOT EXISTS questions 
(queation_id int PRIMARY KEY, 
question text)
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS answers 
(respondent_id INTEGER PRIMARY KEY AUTOINCREMENT, 
if_studied_mansi_at_school text,
if_speak_mansi_at_home text,
if_was_taught_in_mansi text)
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS metadata 
(respondent_id INTEGER PRIMARY KEY AUTOINCREMENT, 
date_of_birth text,
place_of_birth text,
first_language text)
""")

conn.commit()
conn.close()

In [94]:
# Заполняю таблицу (1)
conn = sqlite3.connect('mansi_language.db')
cur = conn.cursor()
qs = ['В каком году вы родились?', 
      'Где Вы родились (город, село и т. п.)?',
      'Является ли мансийским Вашим родным (первым) языком?',
      'Вы изучали мансийский язык в школе?',
      'На каком языке Вы общаетесь дома?',
      'На каком языке Вам преподавали учебные предметы?']


for i in range(1, 7):
    cur.execute("INSERT INTO questions VALUES (?,?)",
                [i, qs[i-1]]) 
    
conn.commit()
conn.close()

Сначала создаем объект, который отвечает за интерфейс работы с базой

In [95]:
db = SQLAlchemy()

# создаем приложение
app = Flask(__name__)

# подключаем нашу базу sqlite:/// - это тип базы, потом просто имя файла, который лежит в той же папке
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///mansi_language.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False 

# указываем базе, какое наше приложение
db.app = app
db.init_app(app)

In [96]:
# описываем таблицы

class Questions(db.Model):

    __tablename__ = "questions"
    
    q_id = db.Column('question_id', db.Integer, primary_key=True)
    q = db.Column('question', db.Text)


class Answers(db.Model):
    __tablename__ = "answers"

    respondent_id = db.Column('respondent_id', db.Integer, primary_key=True)
    study_at_school = db.Column('if_studied_mansi_at_school', db.Text)
    speak_home = db.Column('if_speak_mansi_at_home', db.Text)
    was_taught_at_school = db.Column('if_was_taught_in_mansi', db.Text)
    

class Person(db.Model):
    __tablename__ = "metadata"

    respondent_id = db.Column('respondent_id', db.Integer, primary_key=True)
    date_of_birth = db.Column('date_of_birth', db.Text)
    place_of_birth = db.Column('place_of_birth', db.Text)
    difirst_languageed = db.Column('first_language', db.Text)

#### Код flask-приложения из HW_8.py

In [ ]:

@app.route("/")
def instruction():
    return render_template('main_page.html')


@app.route("/questionnaire")
def quest():
    return render_template('questionnaire.html')


@app.route("/statistics")
def stat():
    return render_template('statistics.html')


@app.route('/questionnaire')
def question_page():
    questions = Questions.query.all()
    return render_template(
        'questionnaire.html',
        questions=questions
    )


@app.route('/process', methods=['get'])
def answer_process():
    if not request.args:
        return redirect(url_for('question_page'))

    date_of_birth = request.args.get('birth')
    place_of_birth = request.args.get('place')
    difirst_languageed = request.args.get('native')
    
    # Профиль пользователя
    user = Person(
        date_of_birth=date_of_birth,
        place_of_birth=place_of_birth,
        difirst_languageed=difirst_languageed
    )
    db.session.add(user)
    db.session.commit()
    db.session.refresh(user)
    
    study_at_school = request.args.get('school')
    speak_home = request.args.get('home')
    was_taught_at_school = request.args.get('classes')
    
    answer = Answers(
        study_at_school=study_at_school, 
        speak_home=speak_home, 
        was_taught_at_school=was_taught_at_school
    )
    db.session.add(answer)
    db.session.commit()
    db.session.refresh(user)
    
    return 'Ваш ответ принят! Спасибо за участие <3'
    

if __name__ == "__main__":
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Dec/2020 02:05:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2020 02:05:19] "GET /questionnaire HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2020 02:05:26] "GET /statistics HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2020 02:05:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2020 02:05:32] "GET /questionnaire HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2020 02:05:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2020 02:05:40] "GET /questionnaire HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2020 02:05:45] "GET /process?birth=1990&place=f&native=yes&school=yes&home=mansi&classes=mansi HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2020 02:05:46] "GET /favicon.ico HTTP/1.1" 404 -


### 2. Добавляю на страницу статистику 

#### 2.1 Строю и сохраняю графики

1. Cтатистика по метаданным: "Топ-5 населенных пунктов, в которых родились респонденты"

In [ ]:
con = sqlite3.connect('mansi_language.db')
cur = con.cursor()

get_metadata = """
SELECT *
FROM metadata 
"""

cur.execute(get_metadata)
metadata = cur.fetchall()

# Строю и сохраняю график
all_cities = [i[2] for i in metadata]
freq_cities = collections.Counter(all_cities).most_common(5)

tag = [elem[0] for elem in freq_cities]
freq = [elem[1] for elem in freq_cities]

fig, ax = plt.subplots()

ax.bar(tag, freq, color='lightblue')

plt.title('Топ-5 населенных пунктов, в которых родились респонденты',
         fontsize=15,
         color='white',
         backgroundcolor='mediumseagreen',
         pad=20)
plt.xlabel('населенный пункт')
plt.ylabel('количество респондентов')
ax.set_facecolor('white')
fig.set_facecolor('white')
fig.set_figwidth(8)    
fig.set_figheight(5)    
fig.savefig('cities_graph.png')

2. Статистика по вопросам анкеты: "На каких языках – русском или мансийском – респонденты говорят дома?"

In [ ]:
con = sqlite3.connect('mansi_language.db')
cur = con.cursor()

get_answers = """
SELECT *
FROM answers 
"""

cur.execute(get_answers)
answers = cur.fetchall()

# Строю и сохраняю график
all_answers = [i[2] for i in answers]
freq_lang = collections.Counter(all_answers).items()
lang = [elem[0] for elem in freq_lang]
freq = [elem[1] for elem in freq_lang]

fig2, ax1 = plt.subplots()
ax1.pie(freq, autopct='%1.1f%%', labels=lang, 
        shadow=True, startangle=70)
plt.title('На каких языках, русском или мансийском, респонденты говорят дома?',
         fontsize=10,
         color='white',
         backgroundcolor='mediumseagreen')
ax1.axis('equal')
fig2.savefig('languages_graph.png')